# Difussion score matching
Working with "robust" posteriors

In [2]:
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt

In [3]:
%config InlineBackend.figure_format = "retina"

## The setting
### Proposition 3.1 of [1]
Suppose $y\vert\boldsymbol\theta$ is a member of the exponential family such that
$$
 p({\bf y} \vert \theta) = \exp(\eta(\boldsymbol\theta)^\intercal r(x) - a(\boldsymbol\theta) + b(x)).
$$

Take $\eta(\boldsymbol\theta) = \boldsymbol\theta$ and a squared exponential prior of the form 
$$
    \pi(\boldsymbol\theta)\propto
    \exp\left(-\frac{1}{2}(\boldsymbol\theta - \boldsymbol\mu)^\intercal\boldsymbol\Sigma^{-1}(\boldsymbol\theta - \boldsymbol\mu)\right)
$$

with
* $\eta: \boldsymbol\Theta \to \mathbb{R}^p$,
* $r: {\cal X} \to \mathbb{R}^p$,
* $a: \boldsymbol\Theta \to \mathbb{R}$, and
* $b: {\cal X} \to \mathbb{R}$

Then the posterior takes the form of a truncated Gaussian of the form
$$
    \pi(\boldsymbol\theta \vert y_{1:t}) \propto
    \left(
        -\frac{1}{2}(\boldsymbol\theta - \boldsymbol\mu_T)^\intercal \boldsymbol\Sigma_T^{-1} (\boldsymbol\theta - \boldsymbol\mu_T)
    \right)
$$
with
* $\boldsymbol\Sigma_T^{-1} = \boldsymbol\Sigma^{-1} + 2\omega T \Lambda_T$
* $\boldsymbol\mu_T = \boldsymbol\Sigma_T (\boldsymbol\Sigma^{-1}\boldsymbol\mu - \omega T \nu_T)$
* $\Lambda_T = \frac{1}{T}\sum_{t=1}^T \Lambda(y_t)$
* $\nu_T = \frac{2}{T} \nu(y_t)$
* $\Lambda(y) = \left( \nabla r^\intercal m m^\intercal \nabla r \right)(y)$
* $\nu(y) = \left( \nabla r^\intercal m m^\intercal \nabla r^\intercal + \nabla\cdot (mm^\intercal)\nabla r_t \right)$
* $m: {\cal X} \to \mathbb{R}^{M\times M}$

# References
* [1] Robust and Scalable Bayesian Online Changepoint Detection: https://arxiv.org/abs/2302.04759